Para correr el programa, apreta el botón para correr todas las celdas, por favor se paciente, funciona :) , luego puedes utilizar las funciones de forma casi instantánea. Si deseas utilizar otra función una vez terminado el flujo, corre la celda donde se llama a la función  ```menu()```.

In [ ]:
print('Por favor espera unos minutos mientras se carga el archivo...')

In [2]:
# Importing required libraries
import pandas as pd
from pandas.io.json import json_normalize
from IPython.display import display_html
import warnings
warnings.filterwarnings("ignore")
# Read JSON file containing tweets data and remove tweets not in English

raw_tweets = pd.read_json(r'farmers-protest-tweets-2021-03-5.json', lines=True)
raw_tweets = raw_tweets[raw_tweets['lang']=='en']

In [3]:
# Normalize 'user' field

users = json_normalize(raw_tweets['user'])
users.drop(['description', 'linkTcourl'], axis=1, inplace=True)
users.rename(columns={'id':'userId', 'url':'profileUrl'}, inplace=True)

# Transform 'raw_tweets' DataFrame

# Add column for 'userId'
user_id = []
for user in raw_tweets['user']:
    uid = user['id']
    user_id.append(uid)
raw_tweets['userId'] = user_id

In [4]:
# Remove less important columns
cols = ['id', 'retweetCount', 'userId', 'url', 'date', 'renderedContent']
tweets = raw_tweets[cols]
tweets.rename(columns={'id':'tweetId', 'url':'tweetUrl'}, inplace=True)

# Convert to DataFrame, remove duplicates and keep only English tweets
tweets = pd.DataFrame(tweets)
tweets.drop_duplicates(subset=['tweetId'], inplace=True)

In [5]:
top_tweets = tweets.nlargest(10, 'retweetCount')

In [6]:
users_tweets = tweets[['tweetId', 'tweetUrl','renderedContent', 'userId']].merge(users[['userId','displayname']], on='userId')
top_users = users_tweets.groupby(['displayname']).size().reset_index(name='tweets').sort_values(by='tweets', ascending=False)

In [7]:
tweets['date'] = tweets['date'].dt.strftime('%m-%d-%Y')

In [8]:
top_days = tweets.groupby(['date']).size().reset_index(name='tweets').sort_values(by='tweets', ascending=False)

In [9]:
def show_table(df):
  display_html(df.head(10))

In [10]:
def menu():
  print('\nArchivo cargado')
  print(
    '''
    Elige el número de la función a ejecutar:
    [ 1 ] : Top 10 tweets
    [ 2 ] : Top 10 users
    [ 3 ] : Top 10 days
    [ 4 ] : Top 10 hashtags
    '''
  )
  input_correct = False
  while not input_correct:
    user_input = input('Ingresa un número: ')
    try:
      value = int(user_input)
      if value < 1 or value > 4:
        print('Error: Ingresa un número entre 1 y 4')
      else:
        input_correct = True
        if value == 1:
          print('Top 10 tweets:')
          show_table(top_tweets)
        elif value == 2:
          print('Top 10 users:')
          show_table(top_users)
        elif value == 3:
          print('Top 10 days:')
          show_table(top_days)
        elif value == 4:
          print('funcion 4')
    except ValueError:
      print('Error: Recuerda ingresar un número')
      continue

In [ ]:
menu()